In [13]:
import mido
TICKS_PER_BEAT = 480
TEMPO = int(mido.bpm2tempo(120))

In [2]:
%run 'get_input_vectors.ipynb'

In [3]:
def get_track_sequences(vector_sequence):
    """Get note sequences for each track from vector sequence output from RNN.

    Args:
       vector_sequence: List of one-hot vectors containing 128 * NUM_TRACKS note_on events, 128 * NUM_TRACKS note_off events,
        and NUM_TIMESHIFTS timeshift events in intervals of 10 ms each.
       
    Returns:
        List of track vector sequences.
    """
    track_sequences = [[] for _ in range(NUM_TRACKS)]
    start_time = 0
    for vector in vector_sequence:
        index = vector.index(1)
        # vector is a time event.
        if index >= NUM_MIDI_PITCHES * 2 * NUM_TRACKS:
            time = (index - NUM_MIDI_PITCHES * 2 * NUM_TRACKS + 1) * TIMESHIFT_LENGTH
            start_time += time
        else:
            # vector is a note_off event.
            if index >= NUM_MIDI_PITCHES * NUM_TRACKS:
                note_type = 'note_off'
                index -= NUM_MIDI_PITCHES * NUM_TRACKS
            # vector is a note_on event.
            else:
                note_type = 'note_on'
            track_num = int(index / NUM_MIDI_PITCHES)
            note = index % NUM_MIDI_PITCHES
            track_sequences[track_num].append({"type": note_type, "note": note, "start_time": start_time})
            time_delay = 0
            
    return track_sequences
    

In [14]:
def get_midi_from_vector_sequence(track_sequences):
    """Get midi from track sequences.

    Args:
       track_sequences: List of vector sequences for each track.
       
    Returns:
        MidiFile object.
    """
    mid = mido.MidiFile()
    tracks = [mido.MidiTrack() for _ in range(NUM_TRACKS)]
    mid.tracks.extend(tracks)

    for i, ts in enumerate(track_sequences):
        prev_start_time = 0
        for event in ts:
            time = event["start_time"] - prev_start_time
            prev_start_time = event["start_time"]
            # I picked a random number for velocity.
            mid.tracks[i].append(mido.Message(event["type"], note=event["note"], velocity=50,
                                              time=int(mido.second2tick(time, TICKS_PER_BEAT, TEMPO))))
    
    return mid

In [15]:
tracks

[[{'note': 72, 'time': 0.2, 'type': 'note_on'},
  {'note': 72, 'time': 0.04, 'type': 'note_off'}],
 [{'note': 65, 'time': 0, 'type': 'note_on'},
  {'note': 65, 'time': 0.21, 'type': 'note_off'}],
 [{'note': 63, 'time': 0, 'type': 'note_on'},
  {'note': 63, 'time': 0.2, 'type': 'note_off'}],
 [{'note': 61, 'time': 0, 'type': 'note_on'},
  {'note': 61, 'time': 0.3, 'type': 'note_off'}]]

In [8]:
vector_sequence = tracks_to_vector_sequence(tracks)

In [9]:
track_sequences = get_track_sequences(vector_sequence)
mid = get_midi_from_vector_sequence(track_sequences)
mid.save('generated_midi.mid')

In [12]:
[msg for msg in mid.tracks[0]]

[<message note_on channel=0 note=72 velocity=50 time=192>,
 <message note_off channel=0 note=72 velocity=50 time=47>]